In [0]:
import os

In [0]:
!pip install pydrive                             # Package to use Google Drive API - not installed in Colab VM by default
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
os.chdir('drive/My Drive/MinutiaeNet-master/CoarseNet/')

OSError: ignored

In [0]:
#
#os.chdir('drive/My Drive/Colab Notebooks/LearnNet/CoarseNet')
os.getcwd()

'/content/drive/My Drive/MinutiaeNet-master/CoarseNet'

In [0]:
!ls 'drive/My Drive/MinutiaeNet-master/CoarseNet'

ls: cannot access 'drive/My Drive/MinutiaeNet-master/CoarseNet': No such file or directory


In [0]:
!pip install q keras==2.1.6

In [0]:
import keras
keras.__version__

Using TensorFlow backend.


'2.1.6'

In [0]:
!pip install tensorflow==1.7.0

In [0]:
from google.colab import auth                    # Other necessary packages
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()                         # Follow prompt in the authorization process
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()

In [0]:
#
#os.chdir('drive/My Drive/Colab Notebooks/LearnNet/CoarseNet')
os.getcwd()


'/content/drive/My Drive/MinutiaeNet-master/CoarseNet'

In [0]:
drive = GoogleDrive(gauth)
your_module = drive.CreateFile({"id": "1z8QvnJ30NzwuJaGsZ9mWR4L4N3Vx15xA"})   # "your_module_file_id" is the part after "id=" in the shareable link
your_module.GetContentFile("MinutiaeNet_utils.py")          # Save the .py module file to Colab VM
import MinutiaeNet_utils 

MinutiaeNet_utils.py:8: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 657, in launch_instance
    app.initialize(argv)
  File "</usr/local/lib/python2.7/dist-packages/decorator.pyc:decorator-gen-121>", line 2, in initialize
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 

In [0]:
drive = GoogleDrive(gauth)
your_module = drive.CreateFile({"id": "1KETV6nYsHUaLcM0ivaTZPclcB1_6OKM1"})   # "your_module_file_id" is the part after "id=" in the shareable link
your_module.GetContentFile("CoarseNet_utils.py")          # Save the .py module file to Colab VM
import CoarseNet_utils 

In [0]:
drive = GoogleDrive(gauth)
your_module = drive.CreateFile({"id": "1gZXLuxihxb-TEPOQrfFAHVITddck8yVQ"})   # "your_module_file_id" is the part after "id=" in the shareable link
your_module.GetContentFile("CoarseNet_model.py")          # Save the .py module file to Colab VM
import CoarseNet_model

In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

  Created wheel for gputil: filename=GPUtil-1.4.0-cp27-none-any.whl size=7411 sha256=8e9d56d2355ec6d112d079588a91b195a91cdc5098bdae424e61105b7763bfe0
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


IndexError: ignored

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

In [0]:
"""Code for FineNet in paper "Robust Minutiae Extractor: Integrating Deep Networks and Fingerprint Domain Knowledge" at ICB 2018
  https://arxiv.org/pdf/1712.09401.pdf

  If you use whole or partial function in this code, please cite paper:

  @inproceedings{Nguyen_MinutiaeNet,
    author    = {Dinh-Luan Nguyen and Kai Cao and Anil K. Jain},
    title     = {Robust Minutiae Extractor: Integrating Deep Networks and Fingerprint Domain Knowledge},
    booktitle = {The 11th International Conference on Biometrics, 2018},
    year      = {2018},
    }
"""

from __future__ import absolute_import
from __future__ import division

import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

from datetime import datetime
from MinutiaeNet_utils import *

from keras import backend as K
from keras.optimizers import SGD, Adam

from CoarseNet_utils import *
from CoarseNet_model import *
import argparse

#parser = argparse.ArgumentParser(description='Minutiae Net')
#parser.add_argument('lr', type=str, default="0.005", help='Setting learning rate')

#parser.add_argument('GPU', type=str, default="0", help='Choosing GPU')

#args = parser.parse_args()
#lr=0.005
#GPU=0

#os.environ["CUDA_VISIBLE_DEVICES"] = args.GPU
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

config = K.tf.ConfigProto(gpu_options=K.tf.GPUOptions(allow_growth=True))
sess = K.tf.Session(config=config)
K.set_session(sess)

#batch_size = 2
batch_size = 2
use_multiprocessing = False
input_size = 400


# Can use multiple folders for training
train_set = ['../Dataset/CoarseNet_train/',]

#validate_set = ['../path/to/your/data/',]
validate_set = ['../Dataset/CoarseNet_test/img_files/']

pretrain_dir = '../Models/CoarseNet.h5'
output_dir = '../output_CoarseNet/' +datetime.now().strftime('%Y%m%d-%H%M%S')
FineNet_dir = '../Models/FineNet.h5'

if __name__ =='__main__':

    output_dir = '../output_CoarseNet/trainResults/' + datetime.now().strftime('%Y%m%d-%H%M%S')
    logging = init_log(output_dir)
#    logging.info("Learning rate = %s", args.lr)
    logging.info("Learning rate = %s", 0.005)
    logging.info("Pretrain dir = %s", pretrain_dir)

    train(input_shape=(input_size, input_size), train_set=train_set, output_dir=output_dir,
          pretrain_dir=pretrain_dir, batch_size=batch_size, test_set=validate_set,
#         learning_config=Adam(lr=float(args.lr), beta_1=0.9, beta_2=0.999, epsilon=1e-08, clipnorm=0.9),
          learning_config=Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, clipnorm=0.9), 
          logging=logging)

In [0]:
import json

with open("/var/log/colab-jupyter.log", "r") as fo:
  for line in fo:
    print(json.loads(line)['msg'])